<a href="https://colab.research.google.com/github/Harshkotkar/RAG/blob/main/Reranking(Cross_encoder).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Cross-encoders concatenate both inputs into a single sequence and process them jointly through a transformer, allowing full token-to-token attention for richer interactions. They output a direct relevance score but require processing each pair anew, making them slower.

###Why Reranking?
We use reranking because initial retrieval with embeddings is fast but not very precise. Reranking with a cross-encoder improves relevance by jointly analyzing the query and documents, which leads to more accurate results for the final LLM response.

###Can you give an example?

You: Sure. Query: "How does reranking improve RAG?" A bi-encoder might rank a doc about "RAG music festivals" high because of keyword overlap, while burying the perfect technical explanation. Reranking uses cross-encoders to score query-document pairs deeply, promoting truly relevant docs to top-5 for the LLM.

###What are the benefits?

You: 10-30% hit rate improvement, noise reduction, and better LLM context. Retrieval is RAG's biggest bottleneck—reranking fixes that without sacrificing scale, since you only rerank a small candidate set.

In [3]:
!pip install -U langchain langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.8/495.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.7
    Uninstalling langchain-1.2.7:
      Successfully uninstalled langchain-1.2.7
ERROR: pip's depen

In [4]:
!pip install langchain_google_genai
!pip install langchain-text-splitter
!pip install pincone
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
ERROR: Could not find a version that satisfies the requirement pincone (from versions: none)
ERROR: No matching distribution found for pincone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 5.9 MB/s eta 0:00:00


In [6]:
!pip install langchain sentence-transformers faiss-cpu
!pip install pypdf

In [8]:
!pip install langchain-text-splitters

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS


In [21]:
loader=TextLoader("mynta_faq.txt")# go to the file and geting ready to read it but it is not reading yet
docs=loader.load()#now it is reading the file

In [22]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
docs=splitter.split_documents(docs)


In [33]:
docs[:2]

[Document(metadata={'source': 'mynta_faq.txt'}, page_content='Why are there different prices for the same product? Is it legal?\nMyntra is an online marketplace platform that enables independent sellers to sell their products to buyers. The prices are solely decided by the sellers, and Myntra does not interfere in the same. There could be a possibility that the same product is sold by different sellers at different prices. Myntra rightfully fulfils all legal compliances of onboarding multiple sellers on its forum as it is a marketplace platform.'),
 Document(metadata={'source': 'mynta_faq.txt'}, page_content="How can I contact any seller?\nMyntra.com is a marketplace on which third-party sellers sell products to customers. To contact a seller or raise any grievance against them, please send a letter with the below address on the envelope and include product page URL so that it can be forwarded to the seller.\n\n\nTo,\n\n\n'Include Seller's name'\n\n\nSeller Mailbox: Contact Seller\n\n\

In [17]:
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("enter api key")

enter api key··········


In [29]:
#embedings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    task_type="retrieval_document",
    output_dimensionality=512
)



In [30]:
db=FAISS.from_documents(docs,embeddings)

In [32]:
list(db.docstore._dict.values())[:2]

[Document(id='00b8fa48-bb5e-4a5d-9fe4-cd725a31898c', metadata={'source': 'mynta_faq.txt'}, page_content='Why are there different prices for the same product? Is it legal?\nMyntra is an online marketplace platform that enables independent sellers to sell their products to buyers. The prices are solely decided by the sellers, and Myntra does not interfere in the same. There could be a possibility that the same product is sold by different sellers at different prices. Myntra rightfully fulfils all legal compliances of onboarding multiple sellers on its forum as it is a marketplace platform.'),
 Document(id='be544d10-3e42-4ec5-a2ee-98d34b970c5e', metadata={'source': 'mynta_faq.txt'}, page_content="How can I contact any seller?\nMyntra.com is a marketplace on which third-party sellers sell products to customers. To contact a seller or raise any grievance against them, please send a letter with the below address on the envelope and include product page URL so that it can be forwarded to the 

In [34]:
retriver=db.as_retriever(search_kwargs={"k":10})

###Cross-Encoder Reranker


In [39]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [40]:
def rerank(query,docs):
  pairs=[[query,doc.page_content] for doc in docs ]
  scores=cross_encoder.predict(pairs)

  scored_docs=list(zip(docs,scores))
  scored_docs=sorted(scored_docs,key=lambda x:x[1],reverse=True)

  return [doc for doc, score in scored_docs[:3]] #top 3 retrival

In [42]:
query="Why are there different prices for the same product? Is it legal?"
# step1: retrval using the bi encoders
retrived_docs=retriver.invoke(query)

#step2: Reranking using cross encoders

top_docs= rerank(query,retrived_docs)

for i , doc in enumerate(top_docs):
  print(f"\n Rank{i+1}:\n",doc.page_content)


 Rank1:
 Why are there different prices for the same product? Is it legal?
Myntra is an online marketplace platform that enables independent sellers to sell their products to buyers. The prices are solely decided by the sellers, and Myntra does not interfere in the same. There could be a possibility that the same product is sold by different sellers at different prices. Myntra rightfully fulfils all legal compliances of onboarding multiple sellers on its forum as it is a marketplace platform.

 Rank2:
 I saw the product at Rs. 1000 but post clicking on the product, there are multiple prices and the size which I want is being sold for Rs. 1600. Why is there a change in price in the product description page?

 Rank3:
 is selected based on the application of a number of parameters and price is only one such parameter. However, once you land on the product display page on the platform for a specific style, you will have access to the price offered by all sellers on the platform for the re